**Load Data**

In [1]:
# load the country codes
from utils.geocode_utils import get_country_code_lookup

codes = get_country_code_lookup()

In [2]:
# load reviews
import kagglehub
import pandas as pd
import numpy as np
import swifter
from timeit import timeit
import os

path = kagglehub.dataset_download("christopheiv/winemagdata130k")
fname = 'winemag-data-130k-v2.csv'
reviews = pd.read_csv(os.path.join(path, fname))
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129971 entries, 0 to 129970
Data columns (total 14 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   Unnamed: 0             129971 non-null  int64  
 1   country                129908 non-null  object 
 2   description            129971 non-null  object 
 3   designation            92506 non-null   object 
 4   points                 129971 non-null  int64  
 5   price                  120975 non-null  float64
 6   province               129908 non-null  object 
 7   region_1               108724 non-null  object 
 8   region_2               50511 non-null   object 
 9   taster_name            103727 non-null  object 
 10  taster_twitter_handle  98758 non-null   object 
 11  title                  129971 non-null  object 
 12  variety                129970 non-null  object 
 13  winery                 129971 non-null  object 
dtypes: float64(1), int64(2), object(11)


In [3]:
# extract the locations to augment
location_cols = ['winery', 'region_1', 'region_2', 'province', 'country']
wineries = np.unique(reviews.winery.dropna())
locations = reviews.query('winery in @wineries')[location_cols].copy().drop_duplicates()
print(f'{locations.shape[0] - len(wineries):,d}', 'duplicate winery names in different locations' )
locations

13,659 duplicate winery names in different locations


,winery,region_1,region_2,province,country
0,Nicosia,Etna,NaN,Sicily & Sardinia,Italy
1,Quinta dos Avidagos,NaN,NaN,Douro,Portugal
2,Rainstorm,Willamette Valley,Willamette Valley,Oregon,US
3,St. Julian,Lake Michigan Shore,NaN,Michigan,US
4,Sweet Cheeks,Willamette Valley,Willamette Valley,Oregon,US
...,...,...,...,...,...
129940,Standish,Mendocino,NaN,California,US
129941,Apriori,Mendocino County,NaN,California,US
129945,Birichino,Santa Ynez Valley,Central Coast,California,US
129947,Feudo Principi di Butera,Terre Siciliane,NaN,Sicily & Sardinia,Italy


**Method to throttle**

In [4]:
def lookup(name):
  return codes[name] if name in codes else None

**Make it fast**

In [5]:
from timeit import default_timer as timer
start = timer()
reviews['code'] = reviews['country'].swifter.apply(lookup)
elapsed = timer() - start
print(f'{reviews.shape[0]:,d}', 'rows augmented in', f'{elapsed:.3f}', 'seconds.', f'{elapsed/reviews.shape[0]*1_000:.5f}', 'ms per row.')

Pandas Apply:   0%|          | 0/129971 [00:00<?, ?it/s]

129,971 rows augmented in 0.213 seconds. 0.00164 ms per row.


### Throttling Logic

In [6]:
rate = 10                       # 10 calls per second

expected = 1000 / (60*rate)     # time to process 1000 rows, in minutes

print(f'processing time for {1000:,d} records is expected to take {expected:.2f} minutes')

processing time for 1,000 records is expected to take 1.67 minutes


In [7]:
from ratelimit import limits, sleep_and_retry

# throttled variant of fast operation
@sleep_and_retry
@limits(calls=10, period=1)  # Adjust rate limits as needed
def throttled(name):
  return lookup(name)

In [8]:
n = 1000
samples = reviews.sample(1000)
start = timer()
samples['code'] = samples['country'].swifter.apply(throttled)
elapsed = timer() - start
print(f'{samples.shape[0]:,d}', 'rows augmented in', f'{elapsed/60:.2f}', 'minutes.', f'{elapsed/samples.shape[0]*1_000:.5f}', 'ms per row.')

Pandas Apply:   0%|          | 0/1000 [00:00<?, ?it/s]

1,000 rows augmented in 1.87 minutes. 112.31971 ms per row.


In [9]:
# optimal use of throttling
round(locations.shape[0] / (60*rate), 1)

50.7

In [10]:
# realized throttling
round((elapsed / n * locations.shape[0]) / 60, 1)

56.9

In [11]:
samples.head()

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery,code
56563,56563,US,This is the richest and most satisfying of the...,NaN,88,30.0,California,Russian River Valley,Sonoma,NaN,NaN,Sebastiani 2010 Pinot Noir (Russian River Valley),Pinot Noir,Sebastiani,US
31455,31455,France,"This is a spicy, ripe wine that has fine acidi...",NaN,88,19.0,Bordeaux,Cadillac Côtes de Bordeaux,NaN,Roger Voss,@vossroger,Château Bel-Air 2014 Cadillac Côtes de Bordeaux,Bordeaux-style Red Blend,Château Bel-Air,FR
21565,21565,Italy,Here is a distinctive interpretation of Barber...,Solneri,87,35.0,Piedmont,Barbera d'Asti Superiore Nizza,NaN,NaN,NaN,Pescaja 2007 Solneri (Barbera d'Asti Superior...,Barbera,Pescaja,IT
118066,118066,US,"The blend is mainly Cabernet Sauvignon, with t...",La Joie,96,450.0,California,Sonoma County,Sonoma,NaN,NaN,Verité 2007 La Joie Red (Sonoma County),Bordeaux-style Red Blend,Verité,US
60748,60748,Italy,"From an excellent vintage, this bright Rosso o...",NaN,89,25.0,Tuscany,Rosso di Montalcino,NaN,NaN,NaN,Castiglion del Bosco 2010 Rosso di Montalcino,Sangiovese Grosso,Castiglion del Bosco,IT


## Tests

In [12]:
import unittest

class CountryCodeLookupTest(unittest.TestCase):

  def test_lookup_valid_country(self):
    self.assertEqual(lookup('France'), 'FR')
    self.assertEqual(lookup('United States Of America'), 'US')

  def test_lookup_unknown_country(self):
    self.assertIsNone(lookup('United States'))

  def test_overrides(self):
    self.assertEqual(lookup('US'), 'US')



In [13]:
class CountryCodeAugmentationTest(unittest.TestCase):

  def test_codes_exist(self):
    self.assertTrue('code' in samples.columns)
    self.assertEqual(samples['country'].dropna().count(), samples['code'].dropna().count())
    self.assertTrue(np.all(np.vectorize(lookup)(samples['country'].dropna()) == samples['code'].dropna().to_numpy()))


In [14]:
if __name__ == '__main__':
    unittest.main(argv=[''], verbosity=2, exit=False)

# https://hamatti.org/posts/unit-test-your-python-code-in-jupyter-notebooks/
# https://medium.com/@rajeshpillai/api-rate-limiting-2542c2a90b38

test_codes_exist (__main__.CountryCodeAugmentationTest.test_codes_exist) ... ok
test_lookup_unknown_country (__main__.CountryCodeLookupTest.test_lookup_unknown_country) ... ok
test_lookup_valid_country (__main__.CountryCodeLookupTest.test_lookup_valid_country) ... ok
test_overrides (__main__.CountryCodeLookupTest.test_overrides) ... ok

----------------------------------------------------------------------
Ran 4 tests in 0.006s

OK
